# Getting Started

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('data/survey_results_public.csv')

In [5]:
df.shape

(88883, 85)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88883 entries, 0 to 88882
Data columns (total 85 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Respondent              88883 non-null  int64  
 1   MainBranch              88331 non-null  object 
 2   Hobbyist                88883 non-null  object 
 3   OpenSourcer             88883 non-null  object 
 4   OpenSource              86842 non-null  object 
 5   Employment              87181 non-null  object 
 6   Country                 88751 non-null  object 
 7   Student                 87014 non-null  object 
 8   EdLevel                 86390 non-null  object 
 9   UndergradMajor          75614 non-null  object 
 10  EduOther                84260 non-null  object 
 11  OrgSize                 71791 non-null  object 
 12  DevType                 81335 non-null  object 
 13  YearsCode               87938 non-null  object 
 14  Age1stCode              87634 non-null

In [7]:
# pd.set_option('display.max_columns', df.shape[1])  # configure output to show all columns
pd.set_option('display.max_columns', 5)

Now we can load in the schema CSV file too, and find out what each of the columns in the survey results is all about.

In [8]:
schema = pd.read_csv('data/survey_results_schema.csv')
# pd.set_option('display.max_rows', df.shape[0])   # will allow us to see all 85 rows
pd.set_option('display.max_rows', 8)

Note we can't usually see all 85 rows in the schema, but we've tweaked the `display.max_rows` setting above too.

You can use `.head()` and `.tail()` to limit the number of rows if you don't want that many rows in your output.

In [9]:
schema.head(3)

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?


# DataFrame and Series Basics

You can think of how you're accessing a DataFrame with square brackets as being a bit like accessing a dictionary of lists. The keys are like column labels and the lists are like the Series that define the rows.

In [10]:
people = {
    "first": ["Corey", "Jane"],
    "last": ["Schafer", "Doe"]
}

people["first"]

['Corey', 'Jane']

In [11]:
pd.DataFrame(people)

,first,last
0,Corey,Schafer
1,Jane,Doe


See the `Snippets.ipynb` notebook for code showing how to access rows and columns.

Let's look at what we can do with the main data set.

In [12]:
df['Hobbyist']

0        Yes
1         No
2        Yes
3         No
        ... 
88879     No
88880     No
88881     No
88882    Yes
Name: Hobbyist, Length: 88883, dtype: object

In [13]:
df.loc[0, 'Hobbyist']  # first row, Hobbyist column

'Yes'

In [14]:
df.loc
df.loc[0:2, 'Hobbyist':'Employment']

,Hobbyist,OpenSourcer,OpenSource,Employment
0,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work"
1,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work"
2,Yes,Never,The quality of OSS and closed source software ...,Employed full-time


# Indexes: set, reset and using

In [15]:
df.set_index('Respondent', inplace=True)  # could also use index_col in read_csv

How do we find the question for a column, given that we have the schema frame?

In [16]:
schema.head(3)

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?


In [17]:
schema.set_index('Column', inplace=True)

In [18]:
schema.loc['MgrIdiot']

QuestionText    How confident are you that your manager knows ...
Name: MgrIdiot, dtype: object

In [19]:
schema.sort_index().head(3)

,QuestionText
Column,
Age,What is your age (in years)? If you prefer not...
Age1stCode,At what age did you write your first line of c...
BetterLife,Do you think people born today will have a bet...


# Filtering

Who makes loads of money writing Python, in one of the countries we're most interested in?

In [20]:
countries = ['United States', 'India', 'United Kingdom', 'Germany', 'Canada']
in_country = df['Country'].isin(countries)

In [21]:
does_python = df['LanguageWorkedWith'].str.contains('Python', na=False)

In [22]:
high_salary = (df['ConvertedComp'] > 70000)

In [23]:
df[high_salary & does_python & in_country].head(3)

,MainBranch,Hobbyist,...,SurveyLength,SurveyEase
Respondent,,,,,
22,I am a developer by profession,Yes,...,Appropriate in length,Easy
26,I am a developer by profession,Yes,...,Appropriate in length,Easy
32,I am a developer by profession,No,...,Appropriate in length,Neither easy nor difficult


In [24]:
df.loc[high_salary, ['Country', 'LanguageWorkedWith', 'ConvertedComp']]

,Country,LanguageWorkedWith,ConvertedComp
Respondent,,,
6,Canada,Java;R;SQL,366420.0
9,New Zealand,Bash/Shell/PowerShell;C#;HTML/CSS;JavaScript;P...,95179.0
13,United States,Bash/Shell/PowerShell;HTML/CSS;JavaScript;PHP;...,90000.0
16,United Kingdom,Bash/Shell/PowerShell;C#;HTML/CSS;JavaScript;T...,455352.0
...,...,...,...
88877,United States,Bash/Shell/PowerShell;C;Clojure;HTML/CSS;Java;...,2000000.0
88878,United States,HTML/CSS;JavaScript;Scala;TypeScript,130000.0
88879,Finland,Bash/Shell/PowerShell;C++;Python,82488.0
88882,Netherlands,C#;HTML/CSS;Java;JavaScript;PHP;Python,588012.0


# Sorting data

Imagine we want to look at survey results by country and salary.

In [25]:
df.sort_values(by='Country', inplace=True)

In [26]:
df[['Country', 'ConvertedComp']].head(5)

,Country,ConvertedComp
Respondent,,
39258,Afghanistan,19152.0
63129,Afghanistan,1000000.0
85715,Afghanistan,NaN
50767,Afghanistan,NaN
2782,Afghanistan,NaN


In [27]:
df.sort_values(
    by=['Country', 'ConvertedComp'],
    ascending=[True, False],
    inplace=True
)

In [28]:
df[['Country', 'ConvertedComp']].head(5)

,Country,ConvertedComp
Respondent,,
63129,Afghanistan,1000000.0
50499,Afghanistan,153216.0
39258,Afghanistan,19152.0
58450,Afghanistan,17556.0
7085,Afghanistan,14364.0


Who are the top 10 most highly paid respondents in Afghanistan, and what do they do?

In [29]:
in_afghanistan = df['Country'] == 'Afghanistan'
df[in_afghanistan].nlargest(10, 'ConvertedComp')[['ConvertedComp', 'Employment', 'EdLevel', 'Student']]

,ConvertedComp,Employment,EdLevel,Student
Respondent,,,,
63129,1000000.0,Employed full-time,I never completed any formal education,"Yes, full-time"
50499,153216.0,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",No
39258,19152.0,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",No
58450,17556.0,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",No
...,...,...,...,...
48436,4464.0,Employed full-time,"Secondary school (e.g. American high school, G...","Yes, part-time"
10746,3996.0,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",No
8149,1596.0,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Yes, full-time"
29736,1116.0,Employed full-time,Primary/elementary school,"Yes, part-time"


In [30]:
df.sort_index(inplace=True)

# Grouping and Aggregating

Aggregation involves taking a set of results and returning a single value that represents them (e.g. mean, mode, median, standard deviation, etc.).

So, what's a typical salary for the developers who answered the survye?

In [31]:
df['ConvertedComp'].head(15)

Respondent
1         NaN
2         NaN
3      8820.0
4     61000.0
       ...   
12        NaN
13    90000.0
14    57060.0
15        NaN
Name: ConvertedComp, Length: 15, dtype: float64

In [32]:
df['ConvertedComp'].median()

57287.0

In [33]:
df.median()

CompTotal        62000.0
ConvertedComp    57287.0
WorkWeekHrs         40.0
CodeRevHrs           4.0
Age                 29.0
dtype: float64

In [34]:
df.describe()

,CompTotal,ConvertedComp,WorkWeekHrs,CodeRevHrs,Age
count,5.594500e+04,5.582300e+04,64503.000000,49790.000000,79210.000000
mean,5.519014e+11,1.271107e+05,42.127197,5.084308,30.336699
std,7.331926e+13,2.841523e+05,37.287610,5.513931,9.178390
min,0.000000e+00,0.000000e+00,1.000000,0.000000,1.000000
25%,2.000000e+04,2.577750e+04,40.000000,2.000000,24.000000
50%,6.200000e+04,5.728700e+04,40.000000,4.000000,29.000000
75%,1.200000e+05,1.000000e+05,44.750000,6.000000,35.000000
max,1.000000e+16,2.000000e+06,4850.000000,99.000000,99.000000


In [35]:
df['ConvertedComp'].count()  # how many people answered question?

55823

In [36]:
df['Hobbyist'].value_counts()

Yes    71257
No     17626
Name: Hobbyist, dtype: int64

In [37]:
schema.loc['SocialMedia']

QuestionText    What social media site do you use the most?
Name: SocialMedia, dtype: object

In [38]:
df['SocialMedia'].value_counts()

Reddit            14374
YouTube           13830
WhatsApp          13347
Facebook          13178
                  ...  
VK ВКонта́кте       603
Weibo 新浪微博           56
Youku Tudou 优酷       21
Hello                19
Name: SocialMedia, Length: 14, dtype: int64

In [39]:
df['SocialMedia'].value_counts(normalize=True)

Reddit            0.170233
YouTube           0.163791
WhatsApp          0.158071
Facebook          0.156069
                    ...   
VK ВКонта́кте     0.007141
Weibo 新浪微博        0.000663
Youku Tudou 优酷    0.000249
Hello             0.000225
Name: SocialMedia, Length: 14, dtype: float64

Might the popularity vary geographically?

In [40]:
df['Country'].value_counts()

United States                       20949
India                                9061
Germany                              5866
United Kingdom                       5737
                                    ...  
Saint Kitts and Nevis                   1
Niger                                   1
Saint Vincent and the Grenadines        1
Timor-Leste                             1
Name: Country, Length: 179, dtype: int64

In [41]:
country_group = df.groupby(['Country'])

In [42]:
country_group.get_group('United Kingdom')['SocialMedia'].value_counts()

Reddit            1199
Twitter            995
WhatsApp           909
Facebook           811
                  ... 
WeChat 微信           11
VK ВКонта́кте        3
Youku Tudou 优酷       1
Weibo 新浪微博           1
Name: SocialMedia, Length: 13, dtype: int64

In [43]:
country_group.get_group('United States')['SocialMedia'].value_counts()

Reddit            5700
Twitter           3468
Facebook          2844
YouTube           2463
                  ... 
VK ВКонта́кте        9
Weibo 新浪微博           8
Hello                2
Youku Tudou 优酷       1
Name: SocialMedia, Length: 14, dtype: int64

In [44]:
country_group.get_group('India')['SocialMedia'].value_counts()

WhatsApp          2990
YouTube           1820
LinkedIn           955
Facebook           841
                  ... 
WeChat 微信            5
VK ВКонта́кте        4
Youku Tudou 优酷       2
Weibo 新浪微博           1
Name: SocialMedia, Length: 14, dtype: int64

In [45]:
country_group['SocialMedia'].value_counts()

Country      SocialMedia             
Afghanistan  Facebook                    15
             YouTube                      9
             I don't use social media     6
             WhatsApp                     4
                                         ..
Zimbabwe     YouTube                      3
             Instagram                    2
             LinkedIn                     2
             Reddit                       1
Name: SocialMedia, Length: 1220, dtype: int64

In [46]:
country_group['SocialMedia'].value_counts().loc['India']

SocialMedia
WhatsApp          2990
YouTube           1820
LinkedIn           955
Facebook           841
                  ... 
WeChat 微信            5
VK ВКонта́кте        4
Youku Tudou 优酷       2
Weibo 新浪微博           1
Name: SocialMedia, Length: 14, dtype: int64

In [47]:
country_group['SocialMedia'].value_counts(normalize=True).loc['United Kingdom']

SocialMedia
Reddit            0.216192
Twitter           0.179409
WhatsApp          0.163902
Facebook          0.146232
                    ...   
WeChat 微信         0.001983
VK ВКонта́кте     0.000541
Weibo 新浪微博        0.000180
Youku Tudou 优酷    0.000180
Name: SocialMedia, Length: 13, dtype: float64

In [48]:
country_group['ConvertedComp'].median()

Country
Afghanistan      6222.0
Albania         10818.0
Algeria          7878.0
Andorra        160931.0
                 ...   
Viet Nam        11892.0
Yemen           11940.0
Zambia           5040.0
Zimbabwe        19200.0
Name: ConvertedComp, Length: 179, dtype: float64

In [49]:
country_group['ConvertedComp'].agg(['median', 'mean'])

,median,mean
Country,,
Afghanistan,6222.0,101953.333333
Albania,10818.0,21833.700000
Algeria,7878.0,34924.047619
Andorra,160931.0,160931.000000
...,...,...
Viet Nam,11892.0,17233.436782
Yemen,11940.0,16909.166667
Zambia,5040.0,10075.375000
Zimbabwe,19200.0,34046.666667


In [50]:
df['LanguageWorkedWith'].str.contains('Python').sum()

36443

In [51]:
# This doesn't work, but is left in to show error
country_group['LanguageWorkedWith'].str.contains('Python').sum()

AttributeError: 'SeriesGroupBy' object has no attribute 'str'

In [ ]:
def count_python_users(series):
    return series.str.contains('Python').sum()

country_group['LanguageWorkedWith'].apply(count_python_users)

Country
Afghanistan     8
Albania        23
Algeria        40
Andorra         0
               ..
Viet Nam       78
Yemen           3
Zambia          4
Zimbabwe       14
Name: LanguageWorkedWith, Length: 179, dtype: int64

What if we want the percentages of people who do Python though?

In [ ]:
country_respondents = df['Country'].value_counts()
country_respondents

United States                       20949
India                                9061
Germany                              5866
United Kingdom                       5737
                                    ...  
Papua New Guinea                        1
Dominica                                1
Saint Vincent and the Grenadines        1
Tonga                                   1
Name: Country, Length: 179, dtype: int64

In [ ]:
country_python_respondents = country_group['LanguageWorkedWith'].apply(lambda x: count_python_users(x))

In [ ]:
country_python_respondents / country_respondents

Afghanistan    0.181818
Albania        0.267442
Algeria        0.298507
Andorra        0.000000
                 ...   
Viet Nam       0.337662
Yemen          0.157895
Zambia         0.333333
Zimbabwe       0.358974
Length: 179, dtype: float64

In [ ]:
python = pd.concat(
    [country_respondents, country_python_respondents],
    axis='columns'
)
python

,Country,LanguageWorkedWith
United States,20949,10083
India,9061,3105
Germany,5866,2451
United Kingdom,5737,2384
...,...,...
Papua New Guinea,1,0
Dominica,1,1
Saint Vincent and the Grenadines,1,0
Tonga,1,0


In [ ]:
python.rename(
    columns={'Country': 'NumRespondents', 'LanguageWorkedWith': 'NumKnowsPython'},
    inplace=True)
python

,NumRespondents,NumKnowsPython
United States,20949,10083
India,9061,3105
Germany,5866,2451
United Kingdom,5737,2384
...,...,...
Papua New Guinea,1,0
Dominica,1,1
Saint Vincent and the Grenadines,1,0
Tonga,1,0


In [ ]:
python['PctKnowsPython'] = python['NumKnowsPython'] / python['NumRespondents']
python

,NumRespondents,NumKnowsPython,PctKnowsPython
United States,20949,10083,0.481312
India,9061,3105,0.342677
Germany,5866,2451,0.417832
United Kingdom,5737,2384,0.415548
...,...,...,...
Papua New Guinea,1,0,0.000000
Dominica,1,1,1.000000
Saint Vincent and the Grenadines,1,0,0.000000
Tonga,1,0,0.000000


In [ ]:
python.sort_values('PctKnowsPython', ascending=False, inplace=True)
python

,NumRespondents,NumKnowsPython,PctKnowsPython
Timor-Leste,1,1,1.0
Sao Tome and Principe,1,1,1.0
Niger,1,1,1.0
Dominica,1,1,1.0
...,...,...,...
Lao People's Democratic Republic,3,0,0.0
Gabon,2,0,0.0
Chad,1,0,0.0
Tonga,1,0,0.0


In [ ]:
python[python['NumRespondents'] > 1000].head(10)

,NumRespondents,NumKnowsPython,PctKnowsPython
United States,20949,10083,0.481312
Canada,3395,1558,0.458910
France,2391,1054,0.440820
Russian Federation,1694,708,0.417946
...,...,...,...
Australia,1903,790,0.415134
Netherlands,1852,767,0.414147
Spain,1604,641,0.399626
Italy,1576,625,0.396574


In [ ]:
python.loc['Japan']

NumRespondents    391.000000
NumKnowsPython    182.000000
PctKnowsPython      0.465473
Name: Japan, dtype: float64

# Cleaning Data

In [53]:
df = pd.read_csv('data/survey_results_public.csv', na_values=['NA', 'Missing'])

In [54]:
df['YearsCode'].head(10)

0      4
1    NaN
2      3
3      3
    ... 
6      6
7      8
8     12
9     12
Name: YearsCode, Length: 10, dtype: object

In [56]:
df['YearsCode'].mean()  # oops, the data is currently strings (see dtype above)

TypeError: can only concatenate str (not "int") to str

In [58]:
# df['YearsCode'].astype(int)  # fails with "cannot convert float NaN to integer"

# so what's in it?
df['YearsCode'].unique()

array(['4', nan, '3', '16', '13', '6', '8', '12', '2', '5', '17', '10',
       '14', '35', '7', 'Less than 1 year', '30', '9', '26', '40', '19',
       '15', '20', '28', '25', '1', '22', '11', '33', '50', '41', '18',
       '34', '24', '23', '42', '27', '21', '36', '32', '39', '38', '31',
       '37', 'More than 50 years', '29', '44', '45', '48', '46', '43',
       '47', '49'], dtype=object)

In [61]:
df['YearsCode'].replace('Less than 1 year', 0, inplace=True)
df['YearsCode'].replace('More than 50 years', 51, inplace=True)
df['YearsCode'].unique()

array(['4', nan, '3', '16', '13', '6', '8', '12', '2', '5', '17', '10',
       '14', '35', '7', 0, '30', '9', '26', '40', '19', '15', '20', '28',
       '25', '1', '22', '11', '33', '50', '41', '18', '34', '24', '23',
       '42', '27', '21', '36', '32', '39', '38', '31', '37', 51, '29',
       '44', '45', '48', '46', '43', '47', '49'], dtype=object)

In [67]:
df['YearsCode'].astype(float)  # use float as int doesn't support mean() with NaN

0        4.0
1        NaN
2        3.0
3        3.0
        ... 
88879    NaN
88880    NaN
88881    NaN
88882    8.0
Name: YearsCode, Length: 88883, dtype: float64